In [86]:
import pandas as pd
from azapi import AZlyrics
import json
import requests
from IPython.display import clear_output


In [3]:
def get_file_contents(filename):
    """ Given a filename,
        return the contents of that file
    """
    try:
        with open(filename, 'r') as f:
            # It's assumed our file contains a single line,
            # with our API key
            return f.read().strip()
    except FileNotFoundError:
        print("'%s' file not found" % filename)

api_key = get_file_contents('lastfm_key')

In [4]:
artists = pd.read_csv("artists.csv")

/Users/jason/Library/Python/3.7/lib/python/site-packages/IPython/core/interactiveshell.py:3553: DtypeWarning: Columns (2,4,6) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [5]:
artists.head()

,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


In [6]:
artists.sort_values(by=['listeners_lastfm'])[:5000]
artists.head()

,mbid,artist_mb,artist_lastfm,country_mb,country_lastfm,tags_mb,tags_lastfm,listeners_lastfm,scrobbles_lastfm,ambiguous_artist
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,United Kingdom,United Kingdom,rock; pop; alternative rock; british; uk; brit...,rock; alternative; britpop; alternative rock; ...,5381567.0,360111850.0,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,United Kingdom,United Kingdom,rock; electronic; alternative rock; british; g...,alternative; alternative rock; rock; indie; el...,4732528.0,499548797.0,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,United States,United States,rock; alternative rock; 80s; 90s; rap; metal; ...,rock; alternative rock; alternative; Funk Rock...,4620835.0,293784041.0,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,United States,Barbados; United States,pop; dance; hip hop; reggae; contemporary r b;...,pop; rnb; female vocalists; dance; Hip-Hop; Ri...,4558193.0,199248986.0,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,United States,United States,turkish; rap; american; hip-hop; hip hop; hiph...,rap; Hip-Hop; Eminem; hip hop; pop; american; ...,4517997.0,199507511.0,False


In [7]:
f = open('artists.json')
data = json.load(f)
az_artists = pd.DataFrame(list(data.keys()),columns =['az_artist'])
az_artists.size

19468

In [84]:
merged = artists.merge(az_artists, how="inner",left_on="artist_mb", right_on="az_artist")
merged = merged[['mbid', 'artist_mb', 'artist_lastfm', 'tags_mb', 'az_artist']]
merged['done'] = False
merged = merged.drop_duplicates(subset=['artist_mb'])
merged.to_csv('artists-merged.csv', index=False);
print(merged.shape)
merged.head(20)

(10611, 6)


,mbid,artist_mb,artist_lastfm,tags_mb,az_artist,done
0,cc197bad-dc9c-440d-a5b5-d52ba2e14234,Coldplay,Coldplay,rock; pop; alternative rock; british; uk; brit...,Coldplay,False
1,a74b1b7f-71a5-4011-9441-d0b5e4122711,Radiohead,Radiohead,rock; electronic; alternative rock; british; g...,Radiohead,False
2,8bfac288-ccc5-448d-9573-c33ea2aa5c30,Red Hot Chili Peppers,Red Hot Chili Peppers,rock; alternative rock; 80s; 90s; rap; metal; ...,Red Hot Chili Peppers,False
3,73e5e69d-3554-40d8-8516-00cb38737a1c,Rihanna,Rihanna,pop; dance; hip hop; reggae; contemporary r b;...,Rihanna,False
4,b95ce3ff-3d05-4e87-9e01-c97b66af13d4,Eminem,Eminem,turkish; rap; american; hip-hop; hip hop; hiph...,Eminem,False
5,164f0d73-1234-4e2c-8743-d77bf2191051,Kanye West,Kanye West,synthpop; pop; american; hip-hop; hip hop; ele...,Kanye West,False
6,5b11f4ce-a62d-471e-81fc-a69a8278c7da,Nirvana,Nirvana,rock; alternative rock; 90s; punk; american; e...,Nirvana,False
12,9c9f1380-2516-4fc9-a3e6-f9f61941d090,Muse,Muse,rock; electronic; synthpop; alternative rock; ...,Muse,False
18,0383dadf-2a4e-4d10-a46a-e9e041da8eb3,Queen,Queen,rock; progressive rock; 70s; 80s; 90s; pop-roc...,Queen,False
22,67f66c07-6e61-4026-ade5-7e782fad3a5d,Foo Fighters,Foo Fighters,rock; alternative rock; american; grunge; hard...,Foo Fighters,False


In [88]:
api = AZlyrics()
song_df = pd.DataFrame(columns=['mbid', 'artist_mb', 'artist_lastfm', 'tags_mb', 'az_artist', 'song', 'lyrics', 'image'])
song_df.to_csv('songs.csv', index=False)

In [87]:
data = pd.read_csv("artists-merged.csv")
song_df = pd.read_csv("songs.csv")
good = []
bad = []

for i,row in data.iterrows():
    if not row['done']:
        api.artist = row['az_artist']
        count = 0
        url = f"http://ws.audioscrobbler.com/2.0/?method=artist.gettoptracks&artist={row['artist_lastfm']}&api_key={api_key}&format=json&limit=40"
        r = requests.get(url = url)
        songs
        try:
            songs = r.json()['toptracks']['track']
        except Exception as e:
            print(e)
            print(api.artist, r.json())
            bad.append(row['az_artist'])
            break
        # print(songs)
        for song in songs:
            api.title = song['name']
            if count >= 10:
                print(api.artist, api.title)
                bad.append(row['az_artist'])
                break
            # print(api.artist, api.title)
            try:
                lyrics = api.getLyrics(sleep=5).replace("\n", " ")
                song_df.loc[len(song_df.index)] = list(row[:-1]) + [song['name'], lyrics, song['image'][2]["#text"]]
            except IndexError:
                count += 1
            except AttributeError:
                count += 1
        song_df.to_csv('songs.csv', index=False)
        data.at[i,'done'] = True
        data.to_csv('artists-merged.csv', index=False)
        good.append(row['az_artist'])
        clear_output(wait=True)
        print(f'{len(good)} successed and {len(bad)} failed')
        if len(bad) <= 100:
            print(bad)



KeyboardInterrupt: 

In [38]:
song_df = pd.read_csv("songs.csv")
song_df

,Unnamed: 0,mbid,artist_mb,artist_lastfm,tags_mb,az_artist,song,lyrics,image
